In [ ]:
import os
import cv2
import shutil


class_dir = r'public_data\train'
images_dir = os.path.join(class_dir, 'images')
labels_dir = os.path.join(class_dir, 'labels')

new_i = 'public_data_dummy/train/images/'
new_l = 'public_data_dummy/train/labels/'

if not os.path.exists(new_i):
    os.makedirs(new_i)
if not os.path.exists(new_l):
    os.makedirs(new_l)

for image_filename in os.listdir(images_dir):
    image_path = os.path.join(images_dir, image_filename)
    image = cv2.imread(image_path)
    label_filename = os.path.splitext(image_filename)[0] + '.txt'
    label_path = os.path.join(labels_dir, label_filename)
    with open(label_path, 'r') as f:
        lines = f.readlines()
    # Create a new list to store the positive instances
    positive_labels = []
    for line in lines:
        label = line.strip().split()
        # If the label is a positive instance (class = 1), process it
        if label[0] == '1':
            line = line.replace('1','5',1)
            x_center, y_center, width, height = map(float, label[1:])
            x_min = int((x_center - width/2) * image.shape[1])
            y_min = int((y_center - height/2) * image.shape[0])
            x_max = int((x_center + width/2) * image.shape[1])
            y_max = int((y_center + height/2) * image.shape[0])
            # filename = fit_for_training +label_filename.split('.txt')[0]+str(i)+'.jpg'
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)
            # Append the positive instance to the list of positive instances
            positive_labels.append(line)
        else:
            continue
    cv2.imshow('image', image)
    key = cv2.waitKey(0)
    if key == ord(' '):
        # Save image
        shutil.copy(image_path, new_i)
        # Save label file
        # Remove the non-positive instances from the original label file
        with open(label_path, 'w') as f:
            f.writelines(positive_labels)
        shutil.copy(label_path, new_l)
    elif key == 27:
        break


cv2.destroyAllWindows()